In [1]:
import pandas as pd

## 1. Extract all compound IDs that belong to a Chemical Probes Set

Compound sets per data source:
```
Bromodomains chemical toolbox, 277
Chemical Probes.org, 15
Gray Laboratory Probes, 240
MLP probes, 14 --> Legacy compound set
Open Science Probes, 213
High-quality chemical probes, 395 --> Turn into a boolean label
Nature Chemical Biology Probes, 19 --> Legacy compound set
opnMe Portal, 219
Probe Miner (suitable probes), 227
Protein methyltransferases chemical toolbox, 278
SGC Probes, 28
Tool Compound Set, 19`
```


In [5]:
compound2compoundset = pd.read_csv("csv_dumps/3/compoundtocompoundset.csv", sep=",", header=0, usecols=['compound_id', 'compoundset_id'])
compoundsets = (
    pd.read_csv(
        "csv_dumps/3/compoundset.csv",
        sep=",",
        header=0,
        usecols=['compoundsetid', 'name', 'source_url'])
    .rename(columns={'name' : 'source'})
)

In [6]:
probes_compoundsetids = [277, 15, 240, 213, 395, 219, 227, 278, 28, 19]

compound2compoundset = (
    compound2compoundset.merge(
        compoundsets[['compoundsetid', 'source', 'source_url']],
        left_on='compoundset_id',
        right_on='compoundsetid',
        how='inner'
    )
    # Select compounds coming from Probes data sources
    .query('compoundset_id in @probes_compoundsetids')
    .drop(['compoundsetid', 'compoundset_id'], axis=1)
)

print(compound2compoundset.shape[0])
compound2compoundset.head()

4562


,compound_id,source,source_url
29961,30108,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...
29962,29511,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...
29963,32473,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...
29964,7226,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...
29965,26115,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...


# 🧐 Question: If probe in different sources, do we duplicate or want the id to be unique? Unique ID

## 2.1 Pair compound IDs with probes IDs

In [7]:
probes = (
    pd.read_csv(
        "csv_dumps/3/probe.csv",
        sep=",",
        header=0,
        usecols=['probeid', 'compound_id', 'control', 'origin_id'],
        dtype={'probeid':'int64', 'control':'category'})
    .rename(columns={'probeid':'probe_id', 'origin_id':'origin'})
)
probes.head(5)

,probe_id,compound_id,origin,control
0,28998,35591,calculated,NaN
1,29013,28478,calculated,NaN
2,26700,45619,calculated,NaN
3,27263,45951,calculated,NaN
4,28219,46613,calculated,NaN


In [8]:
# Controls are read as strings
def str_to_list(X):
    if isinstance(X, str):
        arr = X.strip('[]').split(',')
        arr = [e.strip("''") for e in arr]
        return arr
    return None
    
    
probes['control'] = probes['control'].apply(lambda X: str_to_list(X))

probes = probes.explode('control')

TypeError: unhashable type: 'list'

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas/_libs/hashtable_class_helper.pxi", line 4588, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'list'


In [11]:
# There are 13 compounds not present in the probes dataset
# All coming from 'High-quality chemical probes'

df = compound2compoundset.merge(
    probes,
    on='compound_id',
    how='left'
)

df.head(5)

,compound_id,source,source_url,probe_id,origin,control
0,30108,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...,24043.0,experimental,D-Moses
1,30108,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...,24263.0,experimental,D-Moses
2,30108,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...,28736.0,experimental,D-Moses
3,29511,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...,24042.0,experimental,GSK4028
4,29511,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...,24233.0,experimental,GSK4028


## 2.2 Extract compound name and inchikey

In [12]:
compounds = (
    pd.read_csv(
        "csv_dumps/3/compound.csv",
        sep=",",
        header=0,
        usecols=['compoundid', 'name', 'inchikey'])
    .rename(columns={'compoundid':'compound_id', 'name':'compound_name', 'inchikey':'inchiKey'})
)

In [13]:
df = df.merge(
    compounds,
    on='compound_id',
    how='left'
)

df.head(5)

,compound_id,source,source_url,probe_id,origin,control,compound_name,inchiKey
0,30108,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...,24043.0,experimental,D-Moses,L-Moses,MSFPLTWUFWOKBX-IFXJQAMLSA-N
1,30108,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...,24263.0,experimental,D-Moses,L-Moses,MSFPLTWUFWOKBX-IFXJQAMLSA-N
2,30108,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...,28736.0,experimental,D-Moses,L-Moses,MSFPLTWUFWOKBX-IFXJQAMLSA-N
3,29511,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...,24042.0,experimental,GSK4028,GSK4027,VZAFGXCWAWRULT-UONOGXRCSA-N
4,29511,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...,24233.0,experimental,GSK4028,GSK4027,VZAFGXCWAWRULT-UONOGXRCSA-N


## 3. Extract target associated with probe

In [15]:
# First extract target_id
probes2targets = pd.read_csv("csv_dumps/3/probetobasetarget.csv",
                             sep=",",
                             header=0,
                             usecols=['probe_id', 'basetarget_id'])
                             

df = df.merge(
    probes2targets,
    on='probe_id',
    how='left'
)

Count of probes without an associated target per datasource:
```
MLP Probes                                     114
High-quality chemical probes                    44
Nature Chemical Biology Probes                  39
Chemical Probes.org                             25
Protein methyltransferases chemical toolbox     20
SGC Probes                                      20
Open Science Probes                              3
Gray Laboratory Probes                           2
opnMe Portal                                     1
Probe Miner (suitable probes)                    1
```

In [16]:
# Then extract target name

base_targets = (
    pd.read_csv(
        "csv_dumps/3/basetarget.csv",
        sep=",",
        header=0,
        usecols=['basetargetid', 'name', 'gene_name'])
    .rename(columns={'basetargetid':'basetarget_id', 'name':'target_name', 'gene_name':'target_symbols'})
)

In [17]:
df = df.merge(
    base_targets,
    on='basetarget_id',
    how='left'
)

df.head(10)

,compound_id,source,source_url,probe_id,origin,control,compound_name,inchiKey,basetarget_id,target_name,target_symbols
0,30108,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...,24043.0,experimental,D-Moses,L-Moses,MSFPLTWUFWOKBX-IFXJQAMLSA-N,22470.0,Histone acetyltransferase KAT2B,KAT2B
1,30108,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...,24043.0,experimental,D-Moses,L-Moses,MSFPLTWUFWOKBX-IFXJQAMLSA-N,30906.0,Histone acetyltransferase KAT2A,KAT2A
2,30108,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...,24043.0,experimental,D-Moses,L-Moses,MSFPLTWUFWOKBX-IFXJQAMLSA-N,32009.0,Histone acetyltransferase KAT2A/KAT2B,"KAT2A,KAT2B"
3,30108,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...,24263.0,experimental,D-Moses,L-Moses,MSFPLTWUFWOKBX-IFXJQAMLSA-N,22470.0,Histone acetyltransferase KAT2B,KAT2B
4,30108,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...,24263.0,experimental,D-Moses,L-Moses,MSFPLTWUFWOKBX-IFXJQAMLSA-N,30906.0,Histone acetyltransferase KAT2A,KAT2A
5,30108,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...,24263.0,experimental,D-Moses,L-Moses,MSFPLTWUFWOKBX-IFXJQAMLSA-N,32009.0,Histone acetyltransferase KAT2A/KAT2B,"KAT2A,KAT2B"
6,30108,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...,28736.0,experimental,D-Moses,L-Moses,MSFPLTWUFWOKBX-IFXJQAMLSA-N,22470.0,Histone acetyltransferase KAT2B,KAT2B
7,30108,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...,28736.0,experimental,D-Moses,L-Moses,MSFPLTWUFWOKBX-IFXJQAMLSA-N,30906.0,Histone acetyltransferase KAT2A,KAT2A
8,30108,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...,28736.0,experimental,D-Moses,L-Moses,MSFPLTWUFWOKBX-IFXJQAMLSA-N,32009.0,Histone acetyltransferase KAT2A/KAT2B,"KAT2A,KAT2B"
9,29511,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...,24042.0,experimental,GSK4028,GSK4027,VZAFGXCWAWRULT-UONOGXRCSA-N,22470.0,Histone acetyltransferase KAT2B,KAT2B


In [18]:
# Extract uniprot ID from target

basetarget2target = (
    pd.read_csv(
        "csv_dumps/3/targettobasetarget.csv",
        sep=",",
        header=0,
        usecols=['basetarget_id', 'target_id']))

targets = (
    pd.read_csv(
        "csv_dumps/3/target.csv",
        sep=",",
        header=0,
        usecols=['targetid', 'organism_id', 'uniprotid'])
    .rename(columns={'targetid': 'target_id', 'uniprotid':'uniprot_ids'})
)

In [19]:
df = (
    # Map from basetarget to target
    df.merge(
        basetarget2target,
        on='basetarget_id',
        how='left'
    )
    # Extract organism_id and uniprot_ids from target
    .merge(
        targets,
        on='target_id',
        how='left'
    )
    # Select human as organism (organism_id = [5428, 2832])
    .query('organism_id in [5428, 2832]')
)


In [20]:
# Split and explode the uniprot_ids columns

df['uniprot_id'] = df['uniprot_ids'].apply(lambda X: X.split(','))
df = df.explode('uniprot_id')

In [18]:
df.head()

,compound_id,source,source_url,probe_id,origin,control,compound_name,inchiKey,basetarget_id,target_name,target_symbols,target_id,organism_id,uniprot_ids,uniprot_id
0,5105,Chemical Probes.org,http://www.chemicalprobes.org/,14769.0,experimental,NaN,XMD8-92,QAPAJIZPZGWAND-UHFFFAOYSA-N,15105.0,Mitogen-activated protein kinase 7,MAPK7,66118.0,2832.0,Q13164,Q13164
3,5105,Chemical Probes.org,http://www.chemicalprobes.org/,14769.0,experimental,NaN,XMD8-92,QAPAJIZPZGWAND-UHFFFAOYSA-N,15105.0,Mitogen-activated protein kinase 7,MAPK7,59987.0,2832.0,Q13164,Q13164
4,5105,Chemical Probes.org,http://www.chemicalprobes.org/,14769.0,experimental,NaN,XMD8-92,QAPAJIZPZGWAND-UHFFFAOYSA-N,15105.0,Mitogen-activated protein kinase 7,MAPK7,59988.0,2832.0,Q13164,Q13164
5,5105,Chemical Probes.org,http://www.chemicalprobes.org/,14769.0,experimental,NaN,XMD8-92,QAPAJIZPZGWAND-UHFFFAOYSA-N,15105.0,Mitogen-activated protein kinase 7,MAPK7,59989.0,2832.0,Q13164,Q13164
6,5105,Chemical Probes.org,http://www.chemicalprobes.org/,14769.0,experimental,NaN,XMD8-92,QAPAJIZPZGWAND-UHFFFAOYSA-N,15105.0,Mitogen-activated protein kinase 7,MAPK7,59990.0,2832.0,Q13164,Q13164


## 4. Extract scores

In [21]:
scores = (
    pd.read_csv(
        "csv_dumps/3/compoundtargetscore.csv",
        sep=",",
        header=0,
        usecols=['compound_id', 'basetarget_id', 'percentage', 'score_id'])
    # Filter with only score_ids of interest
    .query('score_id in [1, 5, 6, 7]')
)

df_s = df.merge(
        scores,
        on=['basetarget_id', 'compound_id'],
        how='left'
)

In [22]:
df_s

,compound_id,source,source_url,probe_id,origin,control,compound_name,inchiKey,basetarget_id,target_name,target_symbols,target_id,organism_id,uniprot_ids,uniprot_id,score_id,percentage
0,30108,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...,24043.0,experimental,D-Moses,L-Moses,MSFPLTWUFWOKBX-IFXJQAMLSA-N,22470.0,Histone acetyltransferase KAT2B,KAT2B,100905.0,2832.0,Q92831,Q92831,7.0,50.0
1,30108,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...,24043.0,experimental,D-Moses,L-Moses,MSFPLTWUFWOKBX-IFXJQAMLSA-N,22470.0,Histone acetyltransferase KAT2B,KAT2B,74032.0,2832.0,Q92831,Q92831,7.0,50.0
2,30108,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...,24043.0,experimental,D-Moses,L-Moses,MSFPLTWUFWOKBX-IFXJQAMLSA-N,22470.0,Histone acetyltransferase KAT2B,KAT2B,74033.0,2832.0,Q92831,Q92831,7.0,50.0
3,30108,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...,24043.0,experimental,D-Moses,L-Moses,MSFPLTWUFWOKBX-IFXJQAMLSA-N,22470.0,Histone acetyltransferase KAT2B,KAT2B,74034.0,2832.0,Q92831,Q92831,7.0,50.0
4,30108,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...,24043.0,experimental,D-Moses,L-Moses,MSFPLTWUFWOKBX-IFXJQAMLSA-N,22470.0,Histone acetyltransferase KAT2B,KAT2B,74035.0,2832.0,Q92831,Q92831,7.0,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99164,92343,opnMe Portal,https://opnme.com/,25459.0,experimental,cis-VZ185,VZ185,ZAGCLFXBHOXXEN-JPTLTNPLSA-N,22542.0,Bromodomain-containing protein 7,BRD7,82184.0,2832.0,Q9NPI1,Q9NPI1,7.0,30.0
99165,66654,opnMe Portal,https://opnme.com/,25460.0,experimental,NaN,BI-9740,MNEOBAAFLAQMKF-VKDGWMQASA-N,22702.0,Dipeptidyl peptidase 1,CTSC,100276.0,2832.0,P53634,P53634,7.0,60.0
99166,66654,opnMe Portal,https://opnme.com/,25460.0,experimental,NaN,BI-9740,MNEOBAAFLAQMKF-VKDGWMQASA-N,22702.0,Dipeptidyl peptidase 1,CTSC,75882.0,2832.0,P53634,P53634,7.0,60.0
99167,66654,opnMe Portal,https://opnme.com/,25460.0,experimental,NaN,BI-9740,MNEOBAAFLAQMKF-VKDGWMQASA-N,22702.0,Dipeptidyl peptidase 1,CTSC,75883.0,2832.0,P53634,P53634,7.0,60.0


In [23]:
df_s["probesDrugsScore"] = df_s.apply(lambda X: X.percentage if X.score_id == 7 else None, axis=1)
df_s['probeMinerScore'] = df_s.apply(lambda X: X.percentage if X.score_id == 1 else None, axis=1)
df_s['scoreInCells'] = df_s.apply(lambda X: X.percentage if X.score_id == 5 else None, axis=1)
df_s['scoreInOrganisms'] = df_s.apply(lambda X: X.percentage if X.score_id == 6 else None, axis=1)

In [24]:
# Right now I have duplicated rows due to:
# several data sources evidencing the same assoc +
# exploded uniprot ids +
# I have to coalesce the score rows

In [24]:
df = df_s.copy()

## 5. Extract MoA

In [25]:
moas = (
    pd.read_csv(
        "csv_dumps/3/compoundaction.csv",
        sep=",",
        header=0,
        usecols=['compound_id', 'target_id', 'actiontype_id'])
)

In [26]:
df = df.merge(
        moas,
        on=['target_id', 'compound_id'],
        how='left'
)

In [27]:
df.head()

,compound_id,source,source_url,probe_id,origin,control,compound_name,inchiKey,basetarget_id,target_name,...,organism_id,uniprot_ids,uniprot_id,score_id,percentage,probesDrugsScore,probeMinerScore,scoreInCells,scoreInOrganisms,actiontype_id
0,30108,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...,24043.0,experimental,D-Moses,L-Moses,MSFPLTWUFWOKBX-IFXJQAMLSA-N,22470.0,Histone acetyltransferase KAT2B,...,2832.0,Q92831,Q92831,7.0,50.0,50.0,NaN,NaN,NaN,NaN
1,30108,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...,24043.0,experimental,D-Moses,L-Moses,MSFPLTWUFWOKBX-IFXJQAMLSA-N,22470.0,Histone acetyltransferase KAT2B,...,2832.0,Q92831,Q92831,7.0,50.0,50.0,NaN,NaN,NaN,NaN
2,30108,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...,24043.0,experimental,D-Moses,L-Moses,MSFPLTWUFWOKBX-IFXJQAMLSA-N,22470.0,Histone acetyltransferase KAT2B,...,2832.0,Q92831,Q92831,7.0,50.0,50.0,NaN,NaN,NaN,NaN
3,30108,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...,24043.0,experimental,D-Moses,L-Moses,MSFPLTWUFWOKBX-IFXJQAMLSA-N,22470.0,Histone acetyltransferase KAT2B,...,2832.0,Q92831,Q92831,7.0,50.0,50.0,NaN,NaN,NaN,NaN
4,30108,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...,24043.0,experimental,D-Moses,L-Moses,MSFPLTWUFWOKBX-IFXJQAMLSA-N,22470.0,Histone acetyltransferase KAT2B,...,2832.0,Q92831,Q92831,7.0,50.0,50.0,NaN,NaN,NaN,NaN


## 6. Map to drugId

In [28]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = (SparkSession.builder
    .appName('spark')
    .getOrCreate())

In [29]:

drug_idx = (
    spark.read.parquet('/Users/irene/Documents/dev/pyspark/21.09.5/molecule')
    .select(col('id').alias('drugId'), 'inchiKey')
    .toPandas()
)

In [30]:
df = (
    # Get drugId
    df.merge(
        drug_idx,
        on='inchiKey',
        how='left')
)


df.head()

,compound_id,source,source_url,probe_id,origin,control,compound_name,inchiKey,basetarget_id,target_name,...,uniprot_ids,uniprot_id,score_id,percentage,probesDrugsScore,probeMinerScore,scoreInCells,scoreInOrganisms,actiontype_id,drugId
0,30108,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...,24043.0,experimental,D-Moses,L-Moses,MSFPLTWUFWOKBX-IFXJQAMLSA-N,22470.0,Histone acetyltransferase KAT2B,...,Q92831,Q92831,7.0,50.0,50.0,NaN,NaN,NaN,NaN,NaN
1,30108,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...,24043.0,experimental,D-Moses,L-Moses,MSFPLTWUFWOKBX-IFXJQAMLSA-N,22470.0,Histone acetyltransferase KAT2B,...,Q92831,Q92831,7.0,50.0,50.0,NaN,NaN,NaN,NaN,NaN
2,30108,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...,24043.0,experimental,D-Moses,L-Moses,MSFPLTWUFWOKBX-IFXJQAMLSA-N,22470.0,Histone acetyltransferase KAT2B,...,Q92831,Q92831,7.0,50.0,50.0,NaN,NaN,NaN,NaN,NaN
3,30108,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...,24043.0,experimental,D-Moses,L-Moses,MSFPLTWUFWOKBX-IFXJQAMLSA-N,22470.0,Histone acetyltransferase KAT2B,...,Q92831,Q92831,7.0,50.0,50.0,NaN,NaN,NaN,NaN,NaN
4,30108,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...,24043.0,experimental,D-Moses,L-Moses,MSFPLTWUFWOKBX-IFXJQAMLSA-N,22470.0,Histone acetyltransferase KAT2B,...,Q92831,Q92831,7.0,50.0,50.0,NaN,NaN,NaN,NaN,NaN


In [31]:
df = df.astype({
    'probe_id': 'Int64',
    'basetarget_id': 'Int64',
    'target_id': 'Int64',
    'organism_id': 'Int64',
    'score_id': 'Int64'
})

In [32]:
df.to_csv('output/3/chemprobes_v3_1.tsv', sep='\t', header=True, index=False)

In [33]:
df.iloc[0]

compound_id                                                     30108
source                                  Bromodomains chemical toolbox
source_url          https://www.nature.com/articles/s41467-019-096...
probe_id                                                        24043
origin                                                   experimental
control                                                       D-Moses
compound_name                                                 L-Moses
inchiKey                                  MSFPLTWUFWOKBX-IFXJQAMLSA-N
basetarget_id                                                   22470
target_name                           Histone acetyltransferase KAT2B
target_symbols                                                  KAT2B
target_id                                                      100905
organism_id                                                      2832
uniprot_ids                                                    Q92831
uniprot_id          

## 7. Build final dataset

In [34]:
data = spark.read.csv('output/3/chemprobes_v3_1.tsv', sep='\t', header=True)

All posible URLs
```
+-------------------------------------------+---------------------------------------------------------------+
|source                                     |source_url                                                     |
+-------------------------------------------+---------------------------------------------------------------+
|Probe Miner (suitable probes)              |http://probeminer.icr.ac.uk/#/                                 |
|High-quality chemical probes               |null                                                           |
|Chemical Probes.org                        |http://www.chemicalprobes.org/                                 |
|opnMe Portal                               |https://opnme.com/                                             |
|Bromodomains chemical toolbox              |https://www.nature.com/articles/s41467-019-09672-2#article-info|
|SGC Probes                                 |http://www.thesgc.org/chemical-probes                          |
|Gray Laboratory Probes                     |http://graylab.dana-farber.org/probes.html#                    |
|Protein methyltransferases chemical toolbox|https://www.nature.com/articles/s41467-018-07905-4             |
|Nature Chemical Biology Probes             |http://www.nature.com/nchembio/chemical_probes.html            |
|Open Science Probes                        |http://www.sgc-ffm.uni-frankfurt.de/#!start `

```


In [35]:
data.first()

Row(compound_id='30108', source='Bromodomains chemical toolbox', source_url='https://www.nature.com/articles/s41467-019-09672-2#article-info', probe_id='24043', origin='experimental', control='D-Moses', compound_name='L-Moses', inchiKey='MSFPLTWUFWOKBX-IFXJQAMLSA-N', basetarget_id='22470', target_name='Histone acetyltransferase KAT2B', target_symbols='KAT2B', target_id='100905', organism_id='2832', uniprot_ids='Q92831', uniprot_id='Q92831', score_id='7', percentage='50.0', probesDrugsScore='50.0', probeMinerScore=None, scoreInCells=None, scoreInOrganisms=None, actiontype_id=None, drugId=None)

In [36]:
(data
.filter(col('compound_id') == '888')
).show(10, False, True)

-RECORD 0-----------------------------------------------------------------------------------------
 compound_id      | 888                                                                           
 source           | High-quality chemical probes                                                  
 source_url       | null                                                                          
 probe_id         | 28923                                                                         
 origin           | calculated                                                                    
 control          | null                                                                          
 compound_name    | TGX-221                                                                       
 inchiKey         | CPRAGQJXBLMUEL-UHFFFAOYSA-N                                                   
 basetarget_id    | 25012                                                                         
 target_na

In [37]:
data_m = (data
    # Drop columns that are not needed
    .drop('compound_id', 'basetarget_id', 'target_id', 'organism_id', 'score_id', 'percentage', 'uniprot_ids', 'probe_id')
    # Change Probe Miner source name
    .withColumn('source', when(col('source').contains('Probe Miner'), 'Probe Miner').otherwise(col('source')))
    # Build more complete URLs
    .withColumn(
    'source_url',
    when(col('source_url').contains('probeminer'), concat('source_url', 'uniprot_id'))
    .when(col('source_url').contains('probes.org'), concat(lit('https://new.chemicalprobes.org/?q='), 'compound_name'))
    .when(col('source_url').contains('thesgc'), concat('source_url', lit('/'), 'compound_name'))
    .when(col('source_url').contains('graylab'), lit(None))
    .when(col('source_url').contains('opnme'), concat(lit('https://opnme.com/search-results/'), 'compound_name'))
    .when(col('source_url').contains('frankfurt'),
          concat(
              element_at(split(col('source_url'), '#!start'), 1),
              lit('#!specificprobeoverview/'),
              'compound_name'))
    .otherwise(col('source_url')))
    # Create high-quality flag
    .withColumn('isHighQuality', when(col('source') == 'High-quality chemical probes', True).otherwise(False))
    # Clean rows where control = 1
    .withColumn('control', when(~col('control').contains(1), col('control')))
    .withColumn('control', when(~col('control').contains('2S'), col('control')))
    # Group data and collect sets
    .groupBy('uniprot_id', 'compound_name', 'drugId', 'inchiKey')
    .agg(
        collect_set(
            struct(
                col('source').alias('niceName'),
                col('source_url').alias('url'))
        ).alias('urls'),
        flatten(collect_set(array("control"))).alias('control'), # max size = 1
        collect_set("probesDrugsScore").alias('probesDrugsScore'),
        collect_set("probeMinerScore").alias('probeMinerScore'),
        collect_set("scoreInCells").alias('scoreInCells'),
        collect_set("scoreInOrganisms").alias('scoreInOrganisms'),
        flatten(collect_set(array("actiontype_id"))).alias('mechanismOfAction'),
        flatten(collect_set(array("isHighQuality"))).alias('isHighQuality'),
        collect_set("origin").alias("origin")
    )
    .withColumn('probesDrugsScore', array_max(col('probesDrugsScore')))
    .withColumn('probeMinerScore', array_max(col('probeMinerScore')))
    .withColumn('scoreInCells', array_max(col('scoreInCells')))
    .withColumn('scoreInOrganisms', array_max(col('scoreInOrganisms')))
    .withColumn('mechanismOfAction', expr('filter(mechanismOfAction, x -> x is not null)'))
    .withColumn('control', explode(col('control')))
    .withColumnRenamed('compound_name', 'id')
    .withColumnRenamed('uniprot_id', 'targetFromSourceId')
    .withColumn('urls', array_distinct(col('urls')))
    # Remove reference to High Quality Probes source as this is a subset curated by P&Ds
    # Problem: there are compounds only referenced with High Quality Probes 
    # Solution: remove it if there is already a reference, otherwise reference P&Ds 
    .withColumn('urls', array_except(col('urls'),
                              array(struct(lit('High-quality chemical probes').alias("niceName"), lit(None).cast("string").alias("url")))))
    .withColumn( 
        'urls',
        when(
            size(col('urls')) == 0,
            array(struct(
                lit('Probes & Drugs Portal').alias('niceName'),
                lit('https://www.probes-drugs.org').alias('url')))
        )
        .otherwise(col('urls'))
    )
    # Convert empty arrays into null
    .withColumn('mechanismOfAction', when(size(col('mechanismOfAction')) == 0, lit(None)).otherwise(col('mechanismOfAction')))
    .withColumn('origin', when(size(col('origin')) == 0, lit(None)).otherwise(col('origin')))
    # Transform isHighQuality to keep only True
    .withColumn('isHighQuality', when(array_contains(col('isHighQuality'), True), True).otherwise(False))
    .distinct()
)

data_m.count()

4193

In [38]:
data_m.withColumn('url', explode('urls')).filter(col('url.niceName').contains('Drugs')).first()

Row(targetFromSourceId='O95977', id='ML131', drugId=None, inchiKey='XECIGFKOOKPZGH-UHFFFAOYSA-N', urls=[Row(niceName='Probes & Drugs Portal', url='https://www.probes-drugs.org')], control=None, probesDrugsScore='90.0', probeMinerScore='21.0', scoreInCells=None, scoreInOrganisms=None, mechanismOfAction=None, isHighQuality=True, origin=['experimental'], url=Row(niceName='Probes & Drugs Portal', url='https://www.probes-drugs.org'))

In [39]:
data_m.show(10, False, True)

-RECORD 0--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 targetFromSourceId | P37231                                                                                                                                                                                     
 id                 | PD083636                                                                                                                                                                                   
 drugId             | null                                                                                                                                                                                       
 inchiKey           | DBPCXOFYVLCGCQ-SFHVURJKSA-N                                                                                                               

In [40]:
data_m.coalesce(1).write.json('output/3/chemicalprobes_v3_1')